<a href="https://colab.research.google.com/github/mjmousavi97/MachinLearning-TehranUni-2024/blob/main/Projects/Final%20Project/src/gender_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import glob
import numpy as np
import librosa
import scipy.signal as signal
import gc

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!mkdir '/content/dataset'
!unrar x -o+ "/content/drive/MyDrive/dataset.rar" "/content/dataset"

mkdir: cannot create directory ‘/content/dataset’: File exists

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/dataset.rar

Extracting  /content/dataset/dataset/female/HW1_intro_810100075_female.mp3       0%  OK 
Extracting  /content/dataset/dataset/female/HW1_intro_810101471_female.mp3       0%  OK 
Extracting  /content/dataset/dataset/female/HW1_intro_810102017_female.mp3       0%  OK 
Extracting  /content/dataset/dataset/female/HW1_intro_810102327_female.mp3       0%  OK 
Extracting  /content/dataset/dataset/female/HW1_intro_810103093_female.mp3       0%  OK 
Extracting  /content/dataset/dataset/female/HW1_intro_810103338_female.mp3       0%  OK 
Extracting  /content/dataset/dataset/female/HW1_intro_810600065_female.mp3       0%  OK 
Extracting  /content/dataset/dataset/female/HW1_intro_810600097_female.mp3       0%  OK 
Extracting  /content/data

In [5]:
import os
output_folder = '/content/drive/MyDrive/filtered_audio'
os.makedirs(output_folder, exist_ok=True)

In [6]:
male_dir = '/content/dataset/dataset/male/'
female_dir = '/content/dataset/dataset/female/'

In [7]:
male_file_paths = glob.glob(male_dir + '*.mp3')
female_file_paths = glob.glob(female_dir + '*.mp3')

In [8]:
import scipy.signal as signal

def bandpass_filter(audio, sample_rate, low=300.0, high=3400.0):
    """
    Apply a band-pass Butterworth filter to an audio signal.

    Parameters
    ----------
    audio : numpy.ndarray
        The input audio signal as a 1D NumPy array.
    sr : int or float
        Sampling rate of the audio signal (Hz).
    low : float, optional
        Lower cutoff frequency of the band-pass filter (Hz). Default is 300 Hz.
    high : float, optional
        Upper cutoff frequency of the band-pass filter (Hz). Default is 3400 Hz.

    Returns
    -------
    filtered_audio : numpy.ndarray
        The audio signal after applying the band-pass filter.
    """

    # Calculate Nyquist frequency (half of the sampling rate)
    nyquist = 0.5 * sample_rate

    # Normalize the cutoff frequencies with respect to Nyquist
    low_freq = low / nyquist
    high_freq = high / nyquist

    # Design a 1st-order Butterworth band-pass filter
    b, a = signal.butter(N=1, Wn=[low_freq, high_freq], btype='band')

    # Apply the filter using filtfilt to avoid phase shift
    filtered_audio = signal.filtfilt(b=b, a=a, x=audio)

    return filtered_audio


In [9]:
import os
import librosa
import soundfile as sf
import gc

def process_and_save_audio(file_paths, output_folder, low=300.0, high=3400.0):
    """
    Load, resample, apply band-pass filter, and save audio files one by one to avoid RAM crash.

    Parameters
    ----------
    file_paths : list of str
        List of full paths to audio files.
    output_folder : str
        Folder where filtered audio files will be saved.
    low : float
        Low cutoff frequency of the band-pass filter.
    high : float
        High cutoff frequency of the band-pass filter.
    """

    for file_path in file_paths:
        # Load and resample to 16 kHz
        audio, sr = librosa.load(file_path, sr=16000)

        # Apply band-pass filter
        filtered_audio = bandpass_filter(audio, sr, low, high)

        # Save filtered audio to disk
        output_path = os.path.join(output_folder, os.path.basename(file_path))
        sf.write(output_path, filtered_audio, sr)

        # Free memory
        del audio, filtered_audio
        gc.collect()


In [12]:
process_and_save_audio(male_file_paths,output_folder=output_folder)

In [13]:
process_and_save_audio(female_file_paths, output_folder=output_folder)